In [1]:
!pip install yt_dlp pydub git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3rv76up0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-3rv76up0
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 5.7 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803669 sha256=27b739a445a792aebcb01f43ff06f2ef82b6aed5d818397b81ab4f8a292c1c1b
  Stored in directory: /tmp/pip-ephem-wheel-cache-8mz74w9l/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully

In [2]:
import os
import yt_dlp
import pandas as pd
import whisper
import cv2
import numpy as np
from pydub import AudioSegment
from moviepy.editor import VideoFileClip
from pathlib import Path

In [4]:
# Caminho do dataset
DATASET_PATH = "/kaggle/input/fakenwes-dataset/b5-correlaction-main"
OUTPUT_DIR = "preprocessed_data"

# Criar diretório de saída
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

def extract_frames(video_path, output_folder, fps=1):
    """Extrai frames do vídeo a cada 'fps' segundos usando OpenCV."""
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(frame_rate / fps) if frame_rate > 0 else 1
    count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f'frame_{count:04d}.jpg')
            cv2.imwrite(frame_filename, frame)
        count += 1
    
    cap.release()
    cv2.destroyAllWindows()

def extract_audio(video_path, audio_path):
    """Extrai o áudio do vídeo e converte para WAV usando pydub."""
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    sound = AudioSegment.from_file(audio_path, format="mp4")
    sound.export(audio_path, format="wav")

def transcribe_audio(audio_path):
    """Transcreve o áudio usando Whisper."""
    try:
        model = whisper.load_model("base")
        result = model.transcribe(audio_path)
        return result["text"]
    except AttributeError:
        raise ImportError("Erro ao carregar o modelo Whisper. Verifique a instalação com 'pip install openai-whisper'.")

In [5]:
if __name__ == "__main__":
    video_files = [f for f in os.listdir(DATASET_PATH) if f.endswith(".mp4")]
    
    for video_file in video_files:
        video_path = os.path.join(DATASET_PATH, video_file)
        video_name = os.path.splitext(video_file)[0]
        
        output_video_folder = os.path.join(OUTPUT_DIR, video_name)
        os.makedirs(output_video_folder, exist_ok=True)
        
        audio_path = os.path.join(output_video_folder, "audio.wav")
        frames_folder = os.path.join(output_video_folder, "frames")
        transcription_path = os.path.join(output_video_folder, "transcription.txt")
        
        print(f"Processando vídeo: {video_file}")
        print("Extraindo frames...")
        extract_frames(video_path, frames_folder)
        
        print("Extraindo áudio...")
        extract_audio(video_path, audio_path)
        
        print("Transcrevendo áudio...")
        transcription = transcribe_audio(audio_path)
        with open(transcription_path, "w") as f:
            f.write(transcription)
        
        print(f"Finalizado: {video_file}\n")
    
    print("Pré-processamento concluído!")

Processando vídeo: Video12.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in preprocessed_data/Video12/audio.wav


MoviePy - Done.
Transcrevendo áudio...


100%|███████████████████████████████████████| 139M/139M [00:08<00:00, 17.6MiB/s]


Finalizado: Video12.mp4

Processando vídeo: Video15.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in preprocessed_data/Video15/audio.wav


MoviePy - Done.
Transcrevendo áudio...
Finalizado: Video15.mp4

Processando vídeo: Video7.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in preprocessed_data/Video7/audio.wav


...

MoviePy - Done.
Transcrevendo áudio...
Finalizado: Video14.mp4

Processando vídeo: Video10.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in preprocessed_data/Video10/audio.wav


MoviePy - Done.
Transcrevendo áudio...
Finalizado: Video10.mp4

Processando vídeo: Video8.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in preprocessed_data/Video8/audio.wav


MoviePy - Done.
Transcrevendo áudio...
Finalizado: Video8.mp4

Processando vídeo: Video9.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in preprocessed_data/Video9/audio.wav


MoviePy - Done.
Transcrevendo áudio...
Finalizado: Video9.mp4

Processando vídeo: Video2.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in preprocessed_data/Video2/audio.wav


MoviePy - Done.
Transcrevendo áudio...
Finalizado: Video2.mp4

Pré-processamento concluído!
